<h1> Introduction </h1>

This notebook demonstrates how to scale an implemenation of Full Waveform Inversion using multiple FPGAs with bitstreams containing one or multiple copied compute units.

<h3> Define experiment parameters: </h3>
    
XCLBIN_PATH_DEFAULT => Default path for the .xclbin file containing 1 compute unit  
XCLBIN_PATH_MULTCU => Default path for .xclbin file containing 2 copied compute units  
XRT_ENV_PATH => Path to the Xilinx Runtime setup script.  
DEVICE_NAME_DEFAULT => Default name for the FPGA device  
DIR_PATH => Path to the directory containing the FWI input files.  

In [ ]:
XCLBIN_PATH_DEFAULT = "bitstreams/u280_xclbin/500_500_HBM/FullW.xclbin"
XCLBIN_PATH_MULTCU = "bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin"
XRT_ENV_PATH = "/opt/xilinx/xrt/setup.sh"
DEVICE_NAME_DEFAULT="xilinx_u280_xdma_201920_3"

DIR_PATH = "default/"

<h3> Define the worker method </h3> 

Here, we define the Python method which will be executed on each of the Dask workers. This function calls the driver using the data partition it receives, and returns the output data (along with some performance statistics) to the caller (the Dask client). 

We present two methods, the first can be used to execute single compute unit bitstreams, the second demonstrates how we can use multiple dask workers with the same bitstream.

In [ ]:
def execute_function(grid_data,kernel,id):
    import numpy as np
    import time
    
    start_time = time.time()
    
    # Set up the configuration
    cu = kernel["compute_unit"]
    config = kernel["config"]
    path = kernel["path_to_kernel"]
    grid_data = grid_data
    
    from pynq import Overlay, allocate, Device, lib
    from FWIDriver import FWI

    resolution = config["Freq"]["nTotal"] * config["nSources"] * config["nReceivers"]
    gridsize = len(grid_data)   
    config["tolerance"] = 9.99*10**-7
    config["max"] = 1000

    import socket
    st = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:       
        st.connect(('10.255.255.255', 1))
        IP = st.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        st.close()

    # Load the overlay
    devices = Device.devices
    if len(devices)==0:
        return IP, cu, id
    
    ol = Overlay(path, download=False, device=devices[0])
#     Device.active_device.reset(ol.parser, ol.timestamp, ol.bitfile_name)

    # Allocate the buffers
    A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][0]))
    B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][1]))
    C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,kernel["functions"][0]["dotprod_"+str(cu)][2]))

    D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][0]))
    E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][1]))
    F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,kernel["functions"][1]["update_"+str(cu)][2]))

    # set up the kernel IP's
    dotprod = getattr(ol,"dotprod_"+str(cu))
    update = getattr(ol,"update_"+str(cu))

#     try:

    fwi = FWI(A,B,C,D,E,F,dotprod,update,config,resolution,gridsize,True)

    fwi.pre_process(grid_data)

    # reconstruct the grid by performing Full Wavefrom Inversion
    chi = fwi.reconstruct()

#     except Exception as e:
#         return f"error: {e} ip: {IP} cu {cu}"

    total_time = time.time() - start_time

    # free all the buffers
    A.freebuffer()
    B.freebuffer()
    C.freebuffer()
    D.freebuffer()
    E.freebuffer()
    F.freebuffer()
        
    dict_t = {
    "id": id,
    "cu": cu,
    "time": total_time,
    "chi":chi,
    "IP":IP
    }
    return dict_t
    


<h2> Setup method for multiple compute units </h2>

In [ ]:
def setup_multcu():
    try:
        from pynq import Device, Overlay
        ol = Overlay(XCLBIN_PATH_MULTCU,download=True,device=Device.devices[0])
        print("Overlay downloaded")
    except Exception as e:
        return f" error: {e}"
    return 'setup succesful'
                        

In [ ]:
def tear_down():
    from pynq import Device, Overlay
    try:
        ol = Overlay(XCLBIN_PATH_MULTCU, download = False, device=Device.devices[0])
    except Exception as e:
        return f"error: {e} "
    return "teardown succesful"
        

Second method!

In [ ]:
def run_on_worker(grid_data,kernel,index):
    from dask import delayed, compute
    from pynq import Overlay, allocate, Device, lib
    from multiprocessing import Queue, Process

    devices = Device.devices
    ol = Overlay(kernel["path_to_kernel"], download=True, device=devices[0])
  
    def execute_function(queue,grid_data,kernel,index):
        import numpy as np
        import time    
    
        start_time = time.time()
    
        # Set up the configuration
        cu = kernel["compute_unit"]
        config = kernel["config"]
        path = kernel["path_to_kernel"]
    
        from pynq import Overlay, allocate, Device, lib
        from FWIDriver import FWI
        
        
        resolution = config["Freq"]["nTotal"] * config["nSources"] * config["nReceivers"]
        gridsize = len(grid_data)   
        config["tolerance"] = 9.99*10**-7
        config["max"] = 1000

        # Load the overlay
        devices = Device.devices
        ol = Overlay(path, download=False, device=devices[0])
        
        # Allocate the buffers
        A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=getattr(ol,kernel["functions"]["dotprod_"+index][0]))
        B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,kernel[index]["dotprod_"+index][1]))
        C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,kernel[index]["dotprod_"+index][2]))

        D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,kernel[index]["update_"+index][0]))
        E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,kernel[index]["update_"+index][1]))
        F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,kernel[index]["update_"+index][2]))

        # set up the kernel IP's
        dotprod = getattr(ol,"dotprod_"+index)
        update = getattr(ol,"update_"+index)
        
        fwi = FWI(A,B,C,D,E,F,dotprod,update,config,resolution,gridsize,True)
        
        # pre process the grid data
        fwi.pre_process(grid_data)
        
        # reconstruct the grid by performing Full Wavefrom Inversion
        chi = fwi.reconstruct()
        
        total_time = time.time() - start_time

        # free all the buffers
        A.freebuffer()
        B.freebuffer()
        C.freebuffer()
        D.freebuffer()
        E.freebuffer()
        F.freebuffer()
        
        dict_t = {
        "index": index,
        "time": total_time,
        "chi":chi
        }
        queue.put(dict_t)
    

    # Create a subprocess to handle multiple compute units, if we only use single compute units bitstreams we can map the execute functions directly
    # TODO: example of that?
#     if len(kernel)==1: 
    q = Queue()
    cu = 0
    p = Process(target=execute_function,args=(q, grid_data,kernel,str(cu+1)))
    p.start()
    result = q.get()
    p.join()
#     else:
#         p = []
#         q = []
#         result = []
#         for cu in range(len(kernel)):
#             q.append(Queue())
#             p.append(Process(target=execute_function,args=(q[cu], grid_data[cu],kernel[cu],str(cu+1))))
#             p[cu].start()
#         for cu in range(len(kernel)):
#             result.append(q[cu].get())
#         for cu in range(len(kernel)):
#             p[cu].join()
     
    # Free the overlay and return the results
    ol.free()
    return result
            


<h2> SSH helper function </h2>

https://www.ssh.com/academy/ssh/copy-id

Sometimes we need to connect with a password once. We can use this helper function to login to all the hosts once and use passwordless authentication afterwards.

In [ ]:
import asyncio, asyncssh
import json

with open("cluster_config.json") as f:
    conf = json.load(f)
    
for h in conf["hosts"]:
    async with asyncssh.connect(h,22,password="!",username="") as conn:
        res = await conn.run("uname")
        print(res.stdout,end='')
                

<h2> Set up Octoray </h2>

In [8]:
from Octoray import Octoray

# Create an octoray instance with the 
# octoray = Octoray(ssh_cluster=True, scheduler="10.1.212.126",scheduler_port=8786, hosts=["10.1.212.127","10.1.212.126"], config_file="cluster_config.json")
octoray = Octoray(ssh_cluster=False,config_file="cluster_config.json")


# first load in the data
import json
import copy
import time

# Load in data and config settings
data = []
with open(DIR_PATH+"input/"+"10x10_100"+".txt") as f:
    for l in f:
        data.append(float(l))
        
data.extend(data)

config = None
with open(DIR_PATH+"input/GenericInput.json") as f:
    config = json.load(f)

#set specific configurations for different types of kernels
single_cu_config = config
double_cu_config = copy.deepcopy(config)
single_cu_config["ngrid"]["x"]=50
double_cu_config["ngrid"]["x"]=25

# Configure the kernels by specifying the path to the bitstream, number of compute units, batchsize per compute unit and the function names and variables with their respective memory banks.
single_cu = octoray.create_kernel(XCLBIN_PATH_DEFAULT,1,500,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM3","HBM4","HBM5"]}]],single_cu_config)

double_cu = octoray.create_kernel(XCLBIN_PATH_MULTCU,2,250,[[{"dotprod_1":["HBM0","HBM1","HBM2"]},{"update_1":["HBM6","HBM7","HBM8"]}],
                                                [{"dotprod_2":["HBM3","HBM4","HBM5"]},{"update_2":["HBM9","HBM10","HBM11"]}]],double_cu_config)

# Finally, add the kernels you want to execute
# data_split, kernels_split = octoray.setup_cluster(data,single_cu,copy.deepcopy(single_cu))
data_split, kernels_split = octoray.setup_cluster(data,double_cu)


Initializing OctoRay with client ip: 10.1.212.127
Waiting until workers are set up on remote machines...
Current amount of workers: 2


In [9]:
# f = octoray.client.submit(setup_multcu)
# print(octoray.client.gather([f]))

import time
t = time.time()
result = octoray.execute_hybrid(execute_function,data_split,kernels_split)
print(result)
print(time.time()-t)


1
2
len futures: 2
[{'id': 1, 'cu': 1, 'time': 2.076277017593384, 'chi': array([ 1.37063615e-03, -2.56189991e-03, -6.85210813e-04,  3.76016524e-03,
       -2.27524446e-03,  5.51645002e-04, -3.93738055e-03,  4.94033254e-03,
       -2.19808452e-03,  1.42432885e-03,  1.76794312e-01,  1.73970765e-01,
        1.66634188e-01,  1.76181052e-01,  1.76358733e-01,  1.71555355e-01,
        1.71624206e-01,  1.79837881e-01,  1.73550760e-01,  1.73394419e-01,
        1.94364580e-01,  1.96225317e-01,  1.98734753e-01,  2.01356267e-01,
        2.00818748e-01,  1.99095309e-01,  1.96256552e-01,  1.96567004e-01,
        2.03693366e-01,  1.98948216e-01,  2.39734751e-01,  2.42247599e-01,
        2.44535295e-01,  2.46205616e-01,  2.42935783e-01,  2.47772085e-01,
        2.39710302e-01,  2.33116343e-01,  2.49215698e-01,  2.48328742e-01,
        2.17973860e-01,  2.23275171e-01,  2.23967550e-01,  2.17822984e-01,
        2.20036392e-01,  2.19316418e-01,  2.20003356e-01,  2.15948391e-01,
        2.24095470e-01,  2.

In [ ]:
# Execute the added kernels on the workers.
# print(kernels_split)

# print(data_split)
results = octoray.execute(run_on_worker,data_split,kernels_split,range(len(octoray.hosts)))

print(results)

In [ ]:

# print(len(data_split[1]))

# def x(data, kernels, *args):
#     if len(data) != len(kernels):
#         raise ValueError("data and kernels don't have same dimensions.")
#     futures = []
#     for i,krnl in enumerate(kernels):
#         if isinstance(krnl,dict):
#             print(i+1)
#             futures.append(self.client.submit(func,data[i],krnl,i+1))
#         elif isinstance(krnl,list):
#             for j,k in enumerate(krnl):
#                 print(i+j+1)
#                 futures.append(self.client.submit(func,data[i][j],k,i+j+1))
#     print(self.client.gather(futures))



<h3> Graceful shutdown for OpenSSH version >= 7.9 </h3>

In [ ]:
octoray.shutdown()

<h3> Ungraceful shutdown for OpenSSH version < 7.9 </h3>

In [ ]:
await octoray.fshutdown()

In [7]:
from dask.distributed import Client, progress

client = Client("tcp://10.1.212.127:8786")

# print(len(octoray.kernels))
# print(len(client.scheduler_info()["workers"]))
client

<Client: 'tcp://10.1.212.127:8786' processes=2 threads=2>